In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import sep
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import astropy.io
from astropy.io import fits

rcParams['figure.figsize'] = [10.,8.]

In [ ]:
#read image into standard 2-d numpy array with 2nd image
data = fits.getdata(r"S:\Users\John\Downloads\hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")

In [ ]:
m , s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.imsave('image.png', data, cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

#### Repeat background subtraction from part 1

In [ ]:
#measure a spatially varying background on the image
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

In [ ]:
# get a "global" mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

In [ ]:
#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.imsave('Background.png',bkg_image, cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#evaluate the background noise as 2-d array
bkg_rms = bkg.rms()

In [ ]:
#show background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.imsave('BackgroundNoise.png', bkg_rms, cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#subtract the background
data_sub=data-bkg.back()

Repeat object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
#how many objects were detected?
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
fig.savefig('DetectedObjects.png')

In [ ]:
# available fields
objects.dtype.names

In [ ]:
#Perform circular aperture photometry with a 3 pixel radius at the locations of the objects
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#Show the first 10 objects results
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

## Issue starts here
#### Convert to AB magnitude (prodedure found online)

In [ ]:
import astropy.units as u
import astropy.constants as c
>>> tint = 1000.*u.s
>>> cr_b = ([3000., 100., 15.] * u.ct) / tint
>>> cr_v = ([4000., 90., 25.] * u.ct) / tint
>>> b_i, v_i = u.Magnitude(cr_b), u.Magnitude(cr_v)
>>> b_i, v_i  
<Magnitude [-1.19280314,  2.5       ,  4.55977185] mag(ct / s)>,
 <Magnitude [-1.50514998,  2.61439373,  4.00514998] mag(ct / s)>

In [ ]:
>>> b_i - v_i  
<Magnitude [ 0.31234684, -0.11439373,  0.55462187] mag>

In [ ]:
>>> atm_ext_b, atm_ext_v = 0.12 * u.mag, 0.08 * u.mag
>>> secz = 1./np.cos(45 * u.deg)
>>> b_i0 = b_i - atm_ext_b * secz
>>> v_i0 = v_i - atm_ext_b * secz
>>> b_i0, v_i0  
(<Magnitude [-1.36250876,  2.33029437,  4.39006622] mag(ct / s)>,
 <Magnitude [-1.67485561,  2.4446881 ,  3.83544435] mag(ct / s)>)

In [ ]:
>>> b_ref, v_ref = 17.2 * u.STmag, 17.0 * u.STmag
>>> b_ref, v_ref  
(<Magnitude 17.2 mag(ST)>, <Magnitude 17. mag(ST)>)
>>> zp_b, zp_v = b_ref - b_i0[0], v_ref - v_i0[0]
>>> zp_b, zp_v  
(<Magnitude 18.56250876 mag(s ST / ct)>,
 <Magnitude 18.67485561 mag(s ST / ct)>)

In [ ]:
>>> (0. * u.STmag).to(u.erg/u.s/u.cm**2/u.AA)  
<Quantity 3.63078055e-09 erg / (Angstrom cm2 s)>
>>> (-21.1 * u.STmag).to(u.erg/u.s/u.cm**2/u.AA)  
<Quantity 1. erg / (Angstrom cm2 s)>

In [ ]:
>>> B, V = b_i0 + zp_b, v_i0 + zp_v
>>> B, V  
(<Magnitude [17.2       , 20.89280314, 22.95257499] mag(ST)>,
 <Magnitude [17.        , 21.1195437 , 22.51029996] mag(ST)>)

In [ ]:
>>> V.to(u.ABmag, u.spectral_density(5500.*u.AA))  
<Magnitude [16.99023831, 21.10978201, 22.50053827] mag(AB)>

Make histogram

In [ ]:
histogram = plt.hist(image_data.flatten(), bins='auto')